Imports

In [ ]:
!pip install ffmpeg-python
!pip install av
!pip install -q -U google-generativeai
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate

In [ ]:
import os
import cv2
import time
import torch
import random
import ffmpeg
import warnings
import numpy as np
import pandas as pd
from PIL import Image
from glob import glob
import soundfile as sf
from json import loads,dumps
import matplotlib.pyplot as plt
from scipy.signal import resample
import typing_extensions as typing
from google.generativeai.types import HarmCategory, HarmBlockThreshold

In [ ]:
# Extracting all required ids 

labels_df = pd.read_csv('/kaggle/input/youtube-data/ground_labels_new.csv') 

transcriptions_df = pd.read_csv('/kaggle/input/youtube-data/native-lang-complete-transcriptions.csv')
df = pd.merge(labels_df, transcriptions_df, on='Video Id', how='inner') 


In [ ]:
# Filtering out available ads and those that are either inappropriate or child directed

df = df[(df['Primary Label'] == 'inappropriate') | (df['Primary Label'] == 'appropriate')] 
df['Primary Label'].value_counts().plot(kind='bar') 

In [ ]:
# Extracting video ids and primary labels

video_ids = list(df['Video Id'])
primary_labels = list(df['Primary Label'])
all_transcriptions = list(df['Transcription'])

In [ ]:
# Extracting data from transcripts

transcriptions = []
lengths = []

for (i, id_) in enumerate(video_ids):
    transcriptions.append(all_transcriptions[i].split("chunks")[0])
    lengths.append(len(all_transcriptions[i].split("chunks")[0]))


In [ ]:
import os 

available_ids = os.listdir('/kaggle/input/youtube-data/Ads/Ads') 
len(available_ids) 

In [ ]:
api_keys1 = [] 
api_keys2 = [] 

In [ ]:
import google.generativeai as genai 

In [ ]:
import torch
print(torch.cuda.is_available())

Define Model 

In [ ]:
class LLM_Output(typing.TypedDict):
    label: str
    language: list[str]
    response: str

Prompt 

In [ ]:
prompt =  """"A video can be considered inappropriate for children if it contains physical violence (cartoonish, realistic, or gory), interpersonal violence (bullying, pranks, meanness, belittling, controlling behavior, talking down to others, or manipulation), self-harm or suicide (depictions of harm inflicted on oneself or suicidal thoughts/tendencies), extreme stunts (life-endangering, high-risk activities/challenges that require adult supervision), dangerous products or services (like paintball, airsoft, fireworks, weapons, hunting equipment, graffiti products), scary content (horror, suspense, loud violence, zombies, skeletons, masks, scary clowns, blood, dangerous fire, car crashes, medical procedures or other scary visuals), sexual content (innuendos, sexual behavior, nudity, suggestiveness), intimacy (romantic conduct, couples expressing love, sensual contact, dating, relationships or any expression of love or loss for a significant other,  lyrics about or mentions of love or romance, heartbreak), inappropriate clothing (significant skin exposure or sheer clothing outside of appropriate contexts such as sunbathing, swinmwear,etc), substance use (mention, depictions, or use of alcohol, drugs, tobacco), gambling (betting, gambling, casinos, online or real world casino games, online trading), offensive language (from mild to severe with words such as “stupid” or “idiot”), stereotypes (racial, ethnic, gender, body shaming), crude humor (inappropriate humor, innuendos), weapons (guns, swords, knives, etc but toy guns such as nerf guns are fine), unsuitable media (ads for violent or adult-themed video games, casino games, games with romance or battle or first person-shooter games), fight sports (boxing, wrestling, martial arts), financial content (difference, rolling spot forex, financial spread betting and synonymous products but banking ads are fine), health and beauty services (cosmetic procedures, weight loss, tanning, piercings, tattoos, cosmetics and makeup (no matter their essential benefits) and other products related to external personal care focused on body image but ads on fashion products and essential hygiene products like sunscreen, shampoo, toothpaste, etc. are fine, however, products like sanitary care and hair removal is inappropriate), ads or sponsored content for consumable food or drink products (regardless of nutritional content), where the product is consumed by a person or shown with branding, packaging, or promotional intent are inappropriate; general content like cooking shows, recipes, or scenes where food is simply present or being eaten without promotion are acceptable, pet/animal food products and healthcare, healthcare or medical topics (reproductive health, mental health, substance abuse, eating disorders, pharmaceuticals, health insurance, any nutritional or dietary supplements), sensitive topics (politics, religion, controversial social issues, astrology, occult, paranormal, religious sermons), privacy or safety risks (services or competitions requiring personal information, social networks for strangers, spy services, or video game skin trading), or any explicit depictions or discussions of death or dying. Consider that you are an expert content moderator. Based on the audio transcription, is the video advertisement provided inappropriate for children on this definition? If it is, label it as “inappropriate.” If it is not, label it as “appropriate.” Justify your conclusion. Also, mention what language is being spoken if there is any. 
Make the outputs in JSON format: {“label”: “appropriate” or “inappropriate”, “justification”: “brief explanation of the classification”, “language”: “language detected in the video”}."""

prompt 

Define function

In [ ]:
len(video_ids)

In [ ]:
remaining_ids = 
[
 '5Ww2HbuFbtw',
 'I6PSyvWiD-Y',
 'vOoJVWITR-E',
 '7KkPDl0a6V8',
 'NRk2CH0gIeI',
 'LU1wQ9xP2dI',
 'FbH_LCxiIVM',
 '524IYIf07EE',
 'Ix-mCljuPgI',
 'xFU0M2cl8FM',
 '4XjPFzefWhk',
 '0KRF7JPsQSo',
 'u9PcmUSr3TQ',
 'Ril7ROmazvQ',
 'aAGZzkyAQaY',
 'mHGbPJ2tY90',
 'jbbqZe08g9w',
 'BilSQFcLOiI',
 'B84349ZaHyg',
 '8Un2khGNb3U', 
'eUl9bfFTU3c',
 'cAMHx-m9oh8',
 'zSYP8hKPeVU',
 '9pGJCXS3i5Q',
 'z1m1g4pHg84',
 'AqjEIeYvQPA',
 'ZT2UXcxTjLk',
 'N2VaBxkJfSw',
 '_UFkghfhSa4',
 'qYgly9fnr1Q',
 'YkVSw2RFGmY',
 'Mi1uEmRgJZU',
 'DDL5qspSBgQ',
 'xUmXCrS-Eak',
 'Q1Ra9rvv66E',
 'OkJdnxuvYjA',
 'xlo5a96XFPs',
 'tlXqG-LwdQo',
 'xez5Br-3wyM',
 'Kxr8LSFefvY',
 '6De_zfZRjCg',
 'KrPK2uJC1Lo',
 'aE_MN1Ey2ws',
 'JSXoDeqUmn4',
 'Z1i7JIzXB-o',
 'og44o8EpaaM',
 'ZWwCf8t8mRU',
 'Dg0sl2PtxEY',
 '7dstgdgVUuk',
 'hJMNQDfE3ag',
 '1pByflS74Dw',
 'B4f93zAspX8',
 '6244VKirfjw',
 '4Xo6cUUHbeI',
 'YAxQiXIQVRg',
 'y5yMEcs2TzU',
 'SbaTHhyV7kE',
 'wS-5y9kWcBk'
 ]

In [ ]:
len(remaining_ids)

In [ ]:
ids = []
predicted_labels = []
languages = []
responses = []
ground_truths = []
remaining = []

key_count = 0 

for i in range(len(video_ids)): 

    if i % 2500 == 0: 
            os.environ["API_KEY"] = zain2
            genai.configure(api_key=os.environ["API_KEY"]) 
            model = genai.GenerativeModel("gemini-2.5-flash", system_instruction="You are an expert content moderator.") 

    if video_ids[i] in remaining_ids:

        try:
            contents_of_ad = os.listdir('/kaggle/input/youtube-data/Ads/Ads/' + video_ids[i]) 
            contents_of_ad.remove('audio.mp3') 
            video_path = '/kaggle/input/youtube-data/Ads/Ads/' + video_ids[i] + '/' + contents_of_ad[0] 

            # Upload audio transcription 
            audio = transcriptions[i] 

            # Make inference
            try:
                response = model.generate_content([audio, prompt],
                                                  generation_config=genai.GenerationConfig(
                                                      response_mime_type="application/json",
                                                      response_schema=LLM_Output),
                                                  safety_settings={
                                                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                                                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                                                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                                                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
                                                  })
            except Exception as e:
                print(f"Error making inference: {e}")
                remaining.append(video_ids[i])
                continue

            # Wrap response.text access in try-except
            try:
                print("\nCompleted for video number:", i, "\t", video_ids[i])

                dictionary = loads(response.text)
                print(dictionary)
                
                id_temp = video_ids[i] 
                predicted_label_temp = dictionary['label'] 
                languages_temp = dictionary['language'] 
                response_temp = dictionary['response'] 
                ground_truths_temp = primary_labels[i] 

                ids.append(id_temp)
                predicted_labels.append(predicted_label_temp)
                languages.append(languages_temp)
                responses.append(response_temp)
                ground_truths.append(ground_truths_temp) 
                
            except Exception as e:
                print(f"Error processing response.text: {e}")
                remaining.append(video_ids[i])
                continue

        except Exception as e:
            print(f"Unexpected error: {e}")
            remaining.append(video_ids[i])
            continue

        # Wait a bit to avoid exceeding rate limits
        time.sleep(20)

# At the end, print remaining videos
print("Remaining videos with errors:", remaining)

In [ ]:
remaining_df = pd.DataFrame({'Video Id': remaining}) 
remaining_df.to_csv('remaining.csv', index=False) 

In [ ]:
for i in range(len(responses)): 
    print('True Label: ', ground_truths[i], '\tPrediction: ', predicted_labels[i]) 

In [ ]:
new_df = pd.DataFrame({
    'Video Id': ids,
    'Primary Label': ground_truths,
    'Predicted Label': predicted_labels,
    'Response': responses, 
    'Languages': languages 
})

new_df.head() 

In [ ]:
results_dir = '/kaggle/working/results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [ ]:
new_df.to_csv('/kaggle/working/results/gemini-native_audios_only_3.csv', index=False) 

In [ ]:
# Changing to binary lists 

predictions = [1 if pred == 'inappropriate' else 0 for pred in predicted_labels] 
ground_truths = [1 if label == 'inappropriate' else 0 for label in ground_truths] 

In [ ]:
# Obtaining classification report 
from sklearn.metrics import classification_report 

report = classification_report(ground_truths, predictions) 
print(report) 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ground_truths, predictions)

plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Appropriate', 'Inapproriate'], yticklabels=['Appropriate', 'Inapproriate'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()